# Classificação de renda

## Enunciado

O conjunto de dados "Adult" (Census Income) reúne informações do censo dos EUA de 1994, com 48.842 registros e 14 atributos demográficos e socioeconômicos. Seu objetivo é construir um modelo de classificação binária para prever se a renda atual de um indivíduo excede USD$50.000$.

**Atributos:**

1. Idade (`age`).
2. Horas trabalhadas por semana (`hours-per-week`).
3. Ganho de capital (`capital-gain`).
4. Perda de capital (`capital-loss`).
5. Peso da amostra (`fnlwgt`).
6. Classe de trabalho (`workclass`).
7. Nível de escolaridade (`education`).
8. Estado civil (`marital-status`).
9. Ocupação (`occupation`).
10. Grau de parentesco (`relationship`).
11. Raça (`race`).
12. Sexo (`sex`).
13. País de origem (`native-country`).

Treine dois modelos de classificação, uma _Árvore de Decisão_ e um modelo _Random Forest_. Avalie cada modelo usando Acurácia, Precisão, Recall, e F1-Score. Após a avaliação, gere uma matriz de confusão e um gráfico de importância de atributos.

Como tarefas opcionais:

- **_EDA_**: Examine as distribuições de idade, horas, ganho, etc.. Investigue correlações, e a influência de `education` e `occupation`.

- **_Validação cruzada_**: Aplique e compare _hold-out_, _K-fold_, e _Stratified K-fold_. Discuta o impacto na variância e no viés dos modelos.

- **_Redução de Dimensionalidade_**: Codifique e normalize variáveis. Aplique _PCA_ e avalie o efeito no desempenho dos classificadores

## TODOs

- EDA

  - Examinar distribuições
  - Correlação
    - Matriz de correlação
    - Correlação com `education` e `occupation`

- Validação cruzada

  - Aplicar algoritmos
    - hold-out
    - k-fold
    - stratified k-fold
  - Discutir impacto
    - na variância
    - no viés dos modelos

- Redução de Dimensionalidade
  - Codificar e normalizar variáveis
  - Aplicar PCA
  - Discutir efeito no desempenho
